# Doc2Vecの動作確認（bot_id=11）


実行条件は下記の通りとします。

- すべての教師データのタグをユニークにするため、[ラベル]\_[連番] 形式のタグを付与


- 品詞を落とさないようにする


- DBOWを使用


- feature数は、実質的な教師データのパターン数と整合させる（ユーザーさんによっては、回答ラベル数が必ずしも回答パターン数と合致しないため、ラベル数の２倍程度の値を設定するのを目安とします）


- 学習時の反復回数を [feature数 * 10] に設定


現在テストデータとして用意されているBot（ID=11：benefitone.csv）で動作確認を行います。

## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

## (2) 学習／予測処理

### (2-1) コーパス生成

コーパス（単語が半角スペースで区切られた文字列）生成時、一部の品詞を落とすようにします。

（＝learning.core.nlang.Nlang クラスの仕様に従います）

In [2]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 11
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}

learning_parameter = LearningParameter(attr)

In [3]:
_datasource = Datasource(type='csv')
learning_training_messages = _datasource.learning_training_messages(_bot_id)
questions = np.array(learning_training_messages['question'])
answer_ids = np.array(learning_training_messages['answer_id'])

2017/05/19 PM 03:42:17 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/19 PM 03:42:18 ['./fixtures/question_answers/toyotsu_human.csv']


In [4]:
import MeCab
import mojimoji

class Nlang_naive:
    @classmethod
    def split(self, text):
        tagger = MeCab.Tagger("-u learning/dict/custom.dic")
        tagger.parse('')  # node.surfaceを取得出来るようにするため、空文字をparseする(Python3のバグの模様)
        node = tagger.parseToNode(text)
        word_list = []
        while node:
            features = node.feature.split(",")
            pos = features[0]
            if pos in ["BOS/EOS", "記号"]:
                node = node.next
                continue

            #print(features)
            lemma = node.feature.split(",")[6]

            if lemma == "*":
                lemma = node.surface  #.decode("utf-8")
                
            word_list.append(mojimoji.han_to_zen(lemma))
            node = node.next
        return " ".join(word_list)

    @classmethod
    def batch_split(self, texts):
        splited_texts = []
        for text in texts:
            splited_texts.append(self.split(text))
        return splited_texts

In [5]:
from learning.core.nlang import Nlang

_sentences = np.array(questions)
_separated_sentences = Nlang_naive.batch_split(_sentences)

### (2-2) タグ付け

ラベル毎の教師データ数を制限した学習セットを構築します。

In [6]:
from gensim import models
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

class Doc2VecTrainingSet:
    def __init__(self):
        self.answer_id_count = {}
        self.selected_answer_ids = []
        self.selected_questions = []
        self.selected_separated_sentences = []

    def build(self, questions, answer_ids, separated_sentences):
        for index, answer_id in enumerate(answer_ids):
            if answer_id not in self.answer_id_count.keys():
                self.answer_id_count[answer_id] = 0

            '''
                データをリストに格納
                ラベルはユニークにする必要があるので、
                [answer_id]_[連番] の形式で編集
            '''
            self.selected_answer_ids.append('%04d_%02d' % (
                answer_id,
                self.answer_id_count[answer_id]
            ))
            self.selected_questions.append(questions[index])
            self.selected_separated_sentences.append(separated_sentences[index])
            self.answer_id_count[answer_id] += 1
        
        return self
    
    def get_tagged_document_list(self):
        '''
            ユニークになったコーパス／ラベルから、
            学習セットを生成する
        '''
        tagged_document_list = self.__corpus_to_sentences(
            self.selected_separated_sentences, 
            self.selected_answer_ids
        )

        return tagged_document_list

    def __get_tagged_document(self, sentences, name):
        '''
            models.doc2vecの仕様に従い
            コーパスにタグ付け
        '''
        words = sentences.split(' ')
        return TaggedDocument(words=words, tags=[name])

    def __corpus_to_sentences(self, separated_sentences, answer_ids):
        '''
            TaggedDocumentを生成し、リストに格納
        '''
        tagged_document_list = []
        for idx, (doc, name) in enumerate(zip(separated_sentences, answer_ids)):
            tagged_document = self.__get_tagged_document(doc, name)
            tagged_document_list.append(tagged_document)
            
        return tagged_document_list

In [7]:
'''
    ユニークになったコーパス／ラベルから、
    学習セットを生成する
    
    ラベル数＝88、サンプル数＝7,083
'''
d2v_training_set = Doc2VecTrainingSet()
d2v_training_set.build(questions, answer_ids, _separated_sentences)

### (2-3) 学習処理／モデルのシリアライズ

In [8]:
def doc2vec_model_path(bot_id):
    model_path = 'prototype/better_algorithm/doc2vec.bot%02d.model' % bot_id

    return model_path

def train_by_doc2vec(bot_id, doc2vec_training_set):
    '''
        パラメータ：
         学習時にDBOWを使用する
         featureの数を回答パターン数の２倍（仮決め）に設定
         反復回数をfeature数の10倍に設定
    '''
    sentence_list = doc2vec_training_set.get_tagged_document_list()
    n_pattern = len(doc2vec_training_set.answer_id_count)
    print('train_by_doc2vec: Train data sample=%d, Train data pattern=%d' % (len(sentence_list), n_pattern))

    n_feature = n_pattern * 2
    n_iter = n_feature * 10
    print('train_by_doc2vec: Feature size=%d, Max iteration count=%d' % (n_feature, n_iter))

    # ボキャブラリ生成／学習実行
    model = Doc2Vec(dm=0, size=n_feature, min_count=1, iter=n_iter)
    model.build_vocab(sentence_list)
    ret = model.train(sentence_list)

    '''
        モデル内に保持されているベクトルの数を取得
        （featureの数 [回答パターン数の２倍] と同じであることを確認）
    '''
    if len(model.docvecs) != len(sentence_list):
        raise Exception('train_by_doc2vec: Failed to create document vector')

    # 学習モデルは、ファイルに保存しておく
    model.save(doc2vec_model_path(bot_id))
    print('train_by_doc2vec: document vector size=%d, return=%d' % (len(model.docvecs), ret))

    return ret

In [9]:
'''
    生成された学習セット（タグ付きドキュメント）を
    使用し、学習実行
'''
train_by_doc2vec(_bot_id, d2v_training_set)

train_by_doc2vec: Train data sample=7083, Train data pattern=88
train_by_doc2vec: Feature size=176, Max iteration count=1760
train_by_doc2vec: document vector size=7083, return=60769938


60769938

### (2-4) 予測処理

In [10]:
def predict(word, bot_id):
    '''
        予測処理にかけるコーパスを生成
        （学習セット作成時と同じ関数を使用）
    '''
    corpus = Nlang_naive.split(word).split()

    '''
        コーパスからベクトルを生成し、
        ロードしたモデルから類似ベクトルを検索
    '''
    loaded_model = models.Doc2Vec.load(doc2vec_model_path(bot_id))
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    return corpus, ret

In [11]:
'''
    契約書を見たいのですが（正解＝4683）
'''
predict('契約書を見たいのですが', _bot_id)

(['契約', '書', 'を', '見る', 'たい', 'の', 'です', 'が'],
 [('4683_24', 0.8125563263893127),
  ('4683_12', 0.8092948794364929),
  ('4683_25', 0.7589855194091797),
  ('4683_00', 0.7501294612884521),
  ('4683_21', 0.7027206420898438),
  ('4683_09', 0.6920285224914551),
  ('4683_15', 0.6821205019950867),
  ('4683_03', 0.6742035746574402),
  ('4683_23', 0.6493698358535767),
  ('4683_11', 0.6471184492111206)])

In [12]:
'''
    EXカードを貸してください（正解＝4678）
'''
predict('EXカードを貸してください', _bot_id)

(['ＥＸ', 'カード', 'を', '貸す', 'て', 'くださる'],
 [('4678_2700', 0.8527379035949707),
  ('4678_692', 0.8525393009185791),
  ('4678_2694', 0.7479832172393799),
  ('4678_686', 0.7462894320487976),
  ('4727_11', 0.7335948944091797),
  ('4727_05', 0.7287341356277466),
  ('4678_4220', 0.7135286331176758),
  ('4742_10', 0.7053501605987549),
  ('4678_2218', 0.7035999894142151),
  ('4678_4226', 0.7026973962783813)])

## (3) accuracy 測定

In [13]:
def predict_similarity(separated_sentence):
    corpus = separated_sentence.split()
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    answer_id, similarity = ret[0]
    return corpus, answer_id, similarity

def get_prediction_statistics(separated_sentences, answer_ids):
    '''
        学習セットの質問文をそのまま予測処理にかけて、
        回答を予測
    '''
    statistics = []
    for i, _ in enumerate(separated_sentences):
        sentence = separated_sentences[i]
        preferred_answer_id = answer_ids[i]
        corpus, answer_id, similarity = predict_similarity(separated_sentences[i])
        corpus_len = len(corpus)
        statistics.append((i, corpus_len, preferred_answer_id, answer_id, similarity))

    return statistics

In [14]:
def calculate_accuracy(prediction_statistics):
    '''
        予測結果を、質問文の単語数毎／回答ID毎に統計する
    '''
    ncorrect_by_corpus_len = {}
    nsample_by_corpus_len = {}

    ncorrect = 0
    nsample = 0

    for statistics in prediction_statistics:
        i, corpus_len, preferred_answer_id, answer_id, similarity = statistics

        '''
            質問文の単語数ごとに統計を取る
        '''
        if corpus_len not in nsample_by_corpus_len.keys():
            ncorrect_by_corpus_len[corpus_len] = 0
            nsample_by_corpus_len[corpus_len] = 0
        nsample_by_corpus_len[corpus_len] += 1

        '''
            正解かどうか検査
            （NNNN_nn 形式ラベルの上４桁が一致していれば正解とします）
        '''
        nsample += 1
        if preferred_answer_id[0:4] == answer_id[0:4]:
            ncorrect += 1
            ncorrect_by_corpus_len[corpus_len] += 1
    
    '''
        全体の正解率
    '''
    accuracy = ncorrect / nsample
    print("accuracy=%0.3f (%d/%d)" % (accuracy, ncorrect, nsample))

    '''
        質問文の単語数ごとの統計情報を編集
    '''
    info_by_corpus_len = []
    for k, v in ncorrect_by_corpus_len.items():
        info_by_corpus_len.append((
            k, 
            ncorrect_by_corpus_len[k]/nsample_by_corpus_len[k], 
            ncorrect_by_corpus_len[k], 
            nsample_by_corpus_len[k]
        ))

    '''
        質問文の単語数ごとの正解率をリスト
    '''
    print("Accuracy info by word count...")
    for info in info_by_corpus_len:
        print("word_count=%2d: accuracy=%0.3f (%d/%d)" % (
            info[0], info[1], info[2], info[3]
        ))

    return accuracy

In [15]:
loaded_model = models.Doc2Vec.load(doc2vec_model_path(_bot_id))

selected_separated_sentences = d2v_training_set.selected_separated_sentences
selected_answer_ids = d2v_training_set.selected_answer_ids

In [16]:
prediction_statistics = get_prediction_statistics(selected_separated_sentences, selected_answer_ids)
calculate_accuracy(prediction_statistics)

accuracy=0.968 (6859/7083)
Accuracy info by word count...
word_count= 1: accuracy=0.375 (3/8)
word_count= 2: accuracy=1.000 (40/40)
word_count= 3: accuracy=0.987 (300/304)
word_count= 4: accuracy=0.943 (738/783)
word_count= 5: accuracy=0.968 (1351/1396)
word_count= 6: accuracy=0.971 (1183/1218)
word_count= 7: accuracy=0.967 (1120/1158)
word_count= 8: accuracy=0.972 (811/834)
word_count= 9: accuracy=0.984 (506/514)
word_count=10: accuracy=0.981 (257/262)
word_count=11: accuracy=0.966 (201/208)
word_count=12: accuracy=0.986 (136/138)
word_count=13: accuracy=0.968 (91/94)
word_count=14: accuracy=0.952 (59/62)
word_count=15: accuracy=1.000 (28/28)
word_count=16: accuracy=0.929 (13/14)
word_count=17: accuracy=1.000 (12/12)
word_count=18: accuracy=1.000 (6/6)
word_count=19: accuracy=1.000 (2/2)
word_count=23: accuracy=1.000 (2/2)


0.9683749823521107